## Clip (ViT and Transformer all included)

In [1]:
!/opt/bin/nvidia-smi

Sun May 22 13:39:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#get the data from the drive
id = '1aMRFCTeYpZzudslsKlvQ8U5e8KEVBZFQ'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('multi-label-classification-competition-22.zip')
#unzip the dataset
!unzip multi-label-classification-competition-22.zip

Archive:  multi-label-classification-competition-22.zip
replace COMP5329S1A2Dataset/data/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
import re
import pandas as pd
from io import StringIO
FILENAME = '/content/COMP5329S1A2Dataset/train.csv'
with open(FILENAME) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
    error_num = [4790,14716,14961,29895]
    for i in error_num:
      lines[i] = lines[i][:-3]+lines[i][-2:]
    df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")

In [4]:
FILENAME1 = '/content/COMP5329S1A2Dataset/test.csv'
with open(FILENAME1) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
    
    df1 = pd.read_csv(StringIO(''.join(lines)), escapechar="/")

In [ ]:
from PIL import Image 
import numpy as np

data_name = ['/content/COMP5329S1A2Dataset/data/{}.jpg'.format(i) for i in range(30000)]

image_ls = []
for i in range(30000):
  image = Image.open(data_name[i])
  asl = image.resize((336,336))
  #img = np.asarray(asl)
  image_ls.append(asl)
image_ls = np.array(image_ls)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  if sys.path[0] == '':


In [ ]:
import numpy as np
data_name1 = ['/content/COMP5329S1A2Dataset/data/{}.jpg'.format(i) for i in range(30000,40000)]
from PIL import Image 
image_ls_test = []
for i in range(10000):
  image = Image.open(data_name1[i])
  asl = image.resize((336,336))
  #img = np.asarray(asl)
  image_ls_test.append(asl)
image_ls_test = np.array(image_ls_test)

In [5]:
def a(input):
  if len(input) > 1:
    b = sum([ord(i)-48 for i in input])+9
  else:
    b = ord(input)-48
  return b

def transnum(input):
  return [a(x) for x in input.split()]

train_label = []
for i in df['Labels']:
    train_label.append(transnum(i))

from sklearn.preprocessing import MultiLabelBinarizer
one_hot = MultiLabelBinarizer()
label_onehot = (one_hot.fit_transform(train_label))

In [7]:
import torch

label_onehot_tensor = torch.tensor(label_onehot)
label_onehot_tensor = label_onehot_tensor.to(torch.float32)

In [10]:
'''

import torch

all_text_feature = torch.load('/content/drive/MyDrive/text_g_d.pt').to(torch.float32)
all_image_features = torch.load('/content/drive/MyDrive/image_g_d.pt').to(torch.float32)
test_text_feature = torch.load('/content/drive/MyDrive/test_text.pt').to(torch.float32)
test_image_features = torch.load('/content/drive/MyDrive/test_image.pt').to(torch.float32)

'''

In [ ]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from PIL import Image

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

all_image_features = []
all_text_feature = []

for image , label, caption in zip(image_ls,train_label,df['Caption']):

  image_input = preprocess(image).unsqueeze(0).to(device)
  text_inputs = torch.cat([clip.tokenize(f"{caption}")]).to(device) #生成文字描述
  with torch.no_grad():
    image_features = model.encode_image(image_input)
    
    text_features = model.encode_text(text_inputs)
    

  all_image_features.append(image_features)
  all_text_feature.append(text_features)

In [ ]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from PIL import Image

test_image_features = []
test_text_feature = []

for image , caption in zip(image_ls_test,df1['Caption']):

  image_input = preprocess(image).unsqueeze(0).to(device)
  text_inputs = torch.cat([clip.tokenize(f"{caption}")]).to(device) #生成文字描述
  with torch.no_grad():
    image_features = model.encode_image(image_input)
    
    text_features = model.encode_text(text_inputs)
    

  test_image_features.append(image_features)
  test_text_feature.append(text_features)

In [ ]:
test_image_features = torch.cat(test_image_features).to(torch.float32)
test_text_feature = torch.cat(test_text_feature).to(torch.float32)

all_image_features = torch.cat(all_image_features).to(torch.float32)
all_text_feature = torch.cat(all_text_feature).to(torch.float32)

## Bert

In [ ]:
pip install transformers

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
from transformers import TFBertPreTrainedModel,TFBertMainLayer,BertTokenizer
import tensorflow as tf
from transformers.modeling_tf_utils import (
    TFQuestionAnsweringLoss,
    TFTokenClassificationLoss,
    get_initializer,
    keras_serializable,
    shape_list,
)
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score


def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
    return tokenizer.encode_plus(review, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
                truncation=True
              )
# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label
def encode_examples(ds, limit=-1):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
    
    for (i, row) in enumerate(ds.values):
#     for index, row in ds.iterrows():
#         review = row["text"]
#         label = row["y"]
        review = row[1]
        label = list(row[2:])
        bert_input = convert_example_to_feature(review)
        #print(bert_input)
  
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(label)
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)
    
class TFBertForMultilabelClassification(TFBertPreTrainedModel):

    def __init__(self, config, *inputs, **kwargs):
        super(TFBertForMultilabelClassification, self).__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.bert = TFBertMainLayer(config, name='bert')
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(config.num_labels,
                                                kernel_initializer=get_initializer(config.initializer_range),
                                                name='classifier',
                                                activation='sigmoid')

    def call(self, inputs, **kwargs):
        outputs = self.bert(inputs, **kwargs)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # logits, (hidden_states), (attentions)



def measure_auc(label,pred):
  auc = [roc_auc_score(label[:,i],pred[:,i]) for i in list(range(18))]
  return pd.DataFrame({"label_name":["1","2","3","4","5","6","7","8","9","10","11","13","14","15","16","17","18","19"],"auc":auc})


if __name__ == '__main__': 

    # parameters
    train_path = "/content/train_WZD.csv" # 数据路径
    test_path = "/content/test_WZD.csv" # 数据路径
    model_path = "bert-large-uncased" #模型路径，建议预先下载(https://huggingface.co/bert-base-uncased#)

    # parameters
    max_length = 28
    batch_size = 5
    learning_rate = 1e-5
    number_of_epochs = 5
    num_classes = 18 # 类别数

    # read data
    train_val_data = pd.read_csv(train_path)
    TRAIN_VAL_RATIO = 0.98
    LEN = train_val_data.shape[0]
    SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)
    # train data
    train_data = af
    # val data
    val_data = pd.read_csv(test_path)[2000:]
    # test data
    test_data = pd.read_csv(test_path)[2000:]
    
    # tokenizer
    tokenizer = BertTokenizer.from_pretrained(model_path)
    # train dataset
    ds_train_encoded = encode_examples(train_data).shuffle(47).batch(batch_size)
    # val dataset
    ds_val_encoded = encode_examples(val_data).batch(batch_size)
    # test dataset
    ds_test_encoded = encode_examples(test_data).batch(batch_size)
    
    # model initialization
    model = TFBertForMultilabelClassification.from_pretrained(model_path, num_labels=num_classes)
    # optimizer Adam recommended
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
    # we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
    loss = tf.keras.losses.BinaryCrossentropy()
    metric = tf.keras.metrics.CategoricalAccuracy()
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    # fit model
    bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_val_encoded)
    # evaluate val_set
    pred=model.predict(ds_val_encoded)
    #df_auc = measure_auc(val_data.iloc[:,2:].astype(np.float32).values,pred)
    #print("val set mean column auc:",df_auc)
    #predict test_set

## Feature Extract DNN

In [ ]:
import torch.nn as nn

class FEATURE_DECISION_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 18)
        self.dropout = nn.Dropout(p = 0.5)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = self.dropout(tensor)
        tensor = F.gelu(self.fc2(tensor))
        tensor = self.dropout(tensor)
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

In [ ]:
class FEATURE_DECISION_DNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1536, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = F.gelu(self.fc3(tensor))
        tensor = torch.sigmoid(self.fc4(tensor))
        return tensor

In [ ]:
class DECISION_DECISION_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(18, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

In [ ]:
class PCA_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(908, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = F.gelu(self.fc3(tensor))
        tensor = torch.sigmoid(self.fc4(tensor))
        return tensor

In [ ]:
import torch.nn as nn

class AutoEncoder_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1536, 700)
        self.fc2 = nn.Linear(700, 1536)

    def forward(self, inputs):
        tensor = self.fc1(inputs)
        tensor_hidden = tensor
        tensor = F.gelu(tensor)
        tensor = F.sigmoid(self.fc2(tensor))
        return tensor, tensor_hidden

In [ ]:
class AutoEncoder_DNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(700, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = F.gelu(self.fc3(tensor))
        tensor = torch.sigmoid(self.fc4(tensor))
        return tensor

In [ ]:
class HLO_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 18)
        self.dropout = nn.Dropout(p = 0.5)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = self.dropout(tensor)
        tensor = self.fc2(tensor)
        tensor_hidden = tensor
        tensor = F.gelu(tensor)
        tensor = self.dropout(tensor)
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor, tensor_hidden

In [ ]:
class HLO_DNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(512, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 18)
        self.dropout = nn.Dropout(p = 0.5)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = self.dropout(tensor)
        tensor = F.gelu(self.fc2(tensor))
        tensor = self.dropout(tensor)
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

In [ ]:
pip install torchmetrics

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
def Trainer(net, epoch, epoch_step_1, epoch_step_2, Data):

    train_data = DataLoader(TensorDataset(Data[:29400], label_onehot_tensor[:29400]), batch_size=30000, shuffle = False)
    val_data = DataLoader(TensorDataset(Data[29400:], label_onehot_tensor[29400:].to(torch.int32)), batch_size=600, shuffle = False)

    import numpy as np
    from tqdm import tqdm
    import torch.nn.functional as F
    from torchmetrics import F1Score
    from torch import optim
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    Net = net.to(dev)
    opti = optim.Adam(Net.parameters(), lr = 1e-3)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(opti, milestones=[epoch_step_1,epoch_step_2], gamma = 0.1)
    f1 = F1Score(num_classes=18).to(dev)

    epoch = epoch
    loss_list = []
    f1_list = []

    for epoch in tqdm(range(epoch), colour = 'GREEN'):
        for data, label in train_data:      
            data, label = data.to(dev), label.to(dev)
            preds = Net(data)        
            loss = F.binary_cross_entropy(preds, label)
            loss.backward()
            opti.step()
            scheduler.step()
            opti.zero_grad()
        loss_list.append(loss)

        Net.eval()
        with torch.autograd.no_grad():
            for data_val, label_val in val_data:
                data_val, label_val = data_val.to(dev), label_val.to(dev)
                predict = Net(data_val)
                f1_score = f1(predict, label_val)
            print('F1 in epoch{} : {:.4f}'.format(epoch, f1_score))
        f1_list.append(f1_score)
    
    return Net, loss_list, f1_list

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
def Trainer2(net, epoch, epoch_step_1, epoch_step_2, Data):

    train_data = DataLoader(TensorDataset(Data[:29400], Data[:29400]), batch_size=30000, shuffle = False)
    val_data = DataLoader(TensorDataset(Data[29400:], Data[29400:].to(torch.int32)), batch_size=600, shuffle = False)

    import numpy as np
    from tqdm import tqdm
    import torch.nn.functional as F
    from torchmetrics import F1Score
    from torch import optim
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    Net = net.to(dev)
    opti = optim.Adam(Net.parameters(), lr = 1e-3)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(opti, milestones=[epoch_step_1,epoch_step_2], gamma = 0.1)
    f1 = F1Score(num_classes=18).to(dev)

    epoch = epoch
    loss_list = []
    f1_list = []

    for epoch in tqdm(range(epoch), colour = 'GREEN'):
        for data, label in train_data:      
            data, label = data.to(dev), label.to(dev)
            preds,Hiddens = Net(data)        
            loss = F.mse_loss(preds, label)
            loss.backward()
            opti.step()
            scheduler.step()
            opti.zero_grad()
        loss_list.append(loss)

        Net.eval()
        with torch.autograd.no_grad():
            for data_val, label_val in val_data:
                data_val, label_val = data_val.to(dev), label_val.to(dev)
                predict,Hidden = Net(data_val)
                loss = F.mse_loss(predict,label_val)
            print('loss in epoch{} : {:.4f}'.format(epoch,loss))
    
    return Net, loss_list, f1_list

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
def Trainer3(net, epoch, epoch_step_1, epoch_step_2, Data):

    train_data = DataLoader(TensorDataset(Data[:29400], label_onehot_tensor[:29400]), batch_size=30000, shuffle = False)
    val_data = DataLoader(TensorDataset(Data[29400:], label_onehot_tensor[29400:].to(torch.int32)), batch_size=600, shuffle = False)

    import numpy as np
    from tqdm import tqdm
    import torch.nn.functional as F
    from torchmetrics import F1Score
    from torch import optim
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    Net = net.to(dev)
    opti = optim.Adam(Net.parameters(), lr = 1e-3)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(opti, milestones=[epoch_step_1,epoch_step_2], gamma = 0.1)
    f1 = F1Score(num_classes=18).to(dev)

    epoch = epoch
    loss_list = []
    f1_list = []

    for epoch in tqdm(range(epoch), colour = 'GREEN'):
        for data, label in train_data:      
            data, label = data.to(dev), label.to(dev)
            preds,Hiddens = Net(data)        
            loss = F.binary_cross_entropy(preds, label)
            loss.backward()
            opti.step()
            scheduler.step()
            opti.zero_grad()
        loss_list.append(loss)

        Net.eval()
        with torch.autograd.no_grad():
            for data_val, label_val in val_data:
                data_val, label_val = data_val.to(dev), label_val.to(dev)
                predict,Hidden = Net(data_val)
                f1_score = f1(predict, label_val)
            print('F1 in epoch{} : {:.4f}'.format(epoch, f1_score))
        f1_list.append(f1_score)

    
    return Net, loss_list, f1_list

## FEATURES FUSION

### Flatten Concatenation

In [ ]:
Flatten = torch.concat((all_text_feature,all_image_features),dim=-1)
Net, loss_list, f1_list = Trainer(FEATURE_DECISION_DNN2(), 300, 200, 250, Flatten)
Net.eval()
with torch.autograd.no_grad():
    flatten_fusion = Net(Flatten)

###PCA

In [ ]:
pca_ori = torch.concat((all_text_feature,all_image_features),dim=-1).cpu().numpy()

from sklearn.decomposition import PCA
X_train_copy = pca_ori
pca_1 = PCA(n_components=0.98, copy = False)  
X_reduce = pca_1.fit_transform(pca_ori)
n_x = pca_1.n_components

pca=PCA(n_components=n_x, copy = False)
X_reduce_final =pca.fit_transform(X_train_copy)
pca =torch.tensor(X_reduce_final).to(torch.float32)

Net, loss_list, f1_list = Trainer(PCA_DNN(), 300, 200, 250, pca)
Net.eval()
with torch.autograd.no_grad():
    pca_fusion = Net(pca.to(dev))

### AutoEncoder

In [ ]:
Flatten = torch.concat((all_text_feature,all_image_features),dim=-1)
Net, loss_list, f1_list = Trainer2(AutoEncoder_DNN(),300, 220, 280, Flatten)
Net.eval()
with torch.autograd.no_grad():
    auto,auto_hidden = Net(Flatten.to(dev))

In [ ]:
Net, loss_list, f1_list = Trainer(AutoEncoder_DNN2(), 300, 200, 250, auto_hidden)
Net.eval()
with torch.autograd.no_grad():
    auto_fusion = Net(auto_hidden)

###HLO(Hidden Layer Output)

In [ ]:
Net, loss_list, f1_list = Trainer3(HLO_DNN(),300, 195, 250, all_image_features)
Net.eval()
with torch.autograd.no_grad():
    hlo_img,hlo_img_hidden = Net(all_image_features.to(dev))

In [ ]:
Net, loss_list, f1_list = Trainer3(HLO_DNN(),300, 180, 240, all_text_feature)
Net.eval()
with torch.autograd.no_grad():
    hlo_txt,hlo_txt_hidden = Net(all_text_feature.to(dev))

In [ ]:
hlo = hlo_img_hidden+hlo_txt_hidden
Net, loss_list, f1_list = Trainer(HLO_DNN2(),300, 180, 240, hlo)
Net.eval()
with torch.autograd.no_grad():
    hlo_fusion = Net(hlo.to(dev))

## DECISIONS FUSION

### Sum Fusion

In [ ]:
Net, loss_list, f1_list = Trainer(FEATURE_DECISION_DNN(), 300, 195, 250, all_image_features)
Net.eval()
with torch.autograd.no_grad():
    ori_img = Net(all_image_features)

In [ ]:
Net, loss_list, f1_list = Trainer(FEATURE_DECISION_DNN(),300, 180, 240, all_text_feature)
Net.eval()
with torch.autograd.no_grad():
    ori_txt = Net(all_text_feature)

In [ ]:
sum = ori_img+ori_txt
Net, loss_list, f1_list = Trainer(DECISION_DECISION_DNN(),300, 220, 270, sum)
Net.eval()

### Weighted Average fusion

In [ ]:
wa = (0.87*ori_img+0.83*ori_txt)/(0.87+0.83)
Net, loss_list, f1_list = Trainer(DECISION_DECISION_DNN(),300, 220, 270, wa)
Net.eval()
with torch.autograd.no_grad():
    WeightedAvg_fusion = Net(wa)

### Max Fusion

In [ ]:
l = []
for num in range(len(ori_img)):
    s = []
    for i,j in enumerate(zip(ori_img[num], ori_txt[num])):
        s.insert(i, max(j))
    l.append(s)
maxway = torch.tensor(l).to(torch.float32)

Net, loss_list, f1_list = Trainer(DECISION_DECISION_DNN(),300, 220, 270, maxway)
Net.eval()
with torch.autograd.no_grad():
    Max_fusion = Net(maxway.to(dev))


## Mixed Fusion

### Basic Mixed Fusion

In [ ]:
first = flatten_fusion
second = ori_img
third = ori_txt

basicMix = first+second+third
Net, loss_list, f1_list = Trainer(DECISION_DECISION_DNN(),300, 220, 270, basicMix)
Net.eval()
with torch.autograd.no_grad():
    basicMix_fusion = Net(basicMix.to(dev))

### HLO Mixed Fusion

In [ ]:
firsthlo = hlo_fusion
secondhlo = ori_img
thirdhlo = ori_txt

hloMix = firsthlo+secondhlo+thirdhlo
Net, loss_list, f1_list = Trainer(DECISION_DECISION_DNN(),300, 220, 270, hloMix)
Net.eval()
with torch.autograd.no_grad():
    hloMix_fusion = Net(hloMix.to(dev))

## THIS IS THE MODEL!!!!!!!!!!!! LOOK HERE

In [11]:
import torch.nn as nn

class FEATURE_DECISION_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 18)
        self.dropout = nn.Dropout(p = 0.5)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = self.dropout(tensor)
        tensor = F.gelu(self.fc2(tensor))
        tensor = self.dropout(tensor)
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

class DECISION_DECISION_DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(18, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 18)

    def forward(self, inputs):
        tensor = F.gelu(self.fc1(inputs))
        tensor = F.gelu(self.fc2(tensor))
        tensor = torch.sigmoid(self.fc3(tensor))
        return tensor

In [12]:
pip install torchmetrics

     |████████████████████████████████| 409 kB 7.1 MB/s 


In [16]:

import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from torchmetrics import F1Score
from torch import optim

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
def Trainer(net, epoch, epoch_step_1, epoch_step_2, Data):

    train_data = DataLoader(TensorDataset(Data[:29400], label_onehot_tensor[:29400]), batch_size=30000, shuffle = False)
    val_data = DataLoader(TensorDataset(Data[29400:], label_onehot_tensor[29400:].to(torch.int32)), batch_size=600, shuffle = False)

    dev = "cuda" if torch.cuda.is_available() else "cpu"
    Net = net.to(dev)
    opti = optim.Adam(Net.parameters(), lr = 1e-3)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(opti, milestones=[epoch_step_1,epoch_step_2], gamma = 0.1)
    f1 = F1Score(num_classes=18).to(dev)

    epoch = epoch
    loss_list = []
    f1_list = []

    for epoch in tqdm(range(epoch), colour = 'GREEN'):
        for data, label in train_data:      
            data, label = data.to(dev), label.to(dev)
            preds = Net(data)        
            loss = F.binary_cross_entropy(preds, label)
            loss.backward()
            opti.step()
            scheduler.step()
            opti.zero_grad()
        loss_list.append(loss)

        Net.eval()
        with torch.autograd.no_grad():
            for data_val, label_val in val_data:
                data_val, label_val = data_val.to(dev), label_val.to(dev)
                predict = Net(data_val)
                f1_score = f1(predict, label_val)
            print('F1 in epoch{} : {:.4f}'.format(epoch, f1_score))
        f1_list.append(f1_score)
    
    return Net, loss_list, f1_list

In [19]:
dev = "cuda" if torch.cuda.is_available() else "cpu"

Net, loss_list, f1_list = Trainer(FEATURE_DECISION_DNN(), 240, 190, 240, all_image_features)
Net.eval()
with torch.autograd.no_grad():
    img_train = Net(all_image_features.to(dev))
    img_test = Net(test_image_features.to(dev))

In [20]:
Net, loss_list, f1_list = Trainer(FEATURE_DECISION_DNN(), 180, 180, 180, all_text_feature)
Net.eval()
with torch.autograd.no_grad():
    txt_train = Net(all_text_feature.to(dev))
    txt_test = Net(test_text_feature.to(dev))

  0%|          | 1/300 [00:00<01:32,  3.25it/s]

F1 in epoch0 : 0.5782


  1%|          | 2/300 [00:00<01:47,  2.76it/s]

F1 in epoch1 : 0.5778


  1%|          | 3/300 [00:01<01:40,  2.95it/s]

...

F1 in epoch297 : 0.8305


100%|█████████▉| 299/300 [01:40<00:00,  2.95it/s]

F1 in epoch298 : 0.8305


100%|██████████| 300/300 [01:41<00:00,  2.97it/s]

F1 in epoch299 : 0.8305


In [21]:
sum_train = img_train+txt_train
sum_test = img_test+txt_test
Net, loss_list, f1_list = Trainer(DECISION_DECISION_DNN(),300, 220, 270, sum_train)
Net.eval()
with torch.autograd.no_grad():
    final = Net(sum_test)

  0%|          | 1/300 [00:00<02:01,  2.46it/s]

F1 in epoch0 : 0.5587


  1%|          | 2/300 [00:00<01:44,  2.85it/s]

F1 in epoch1 : 0.6187


  1%|          | 3/300 [00:01<01:36,  3.09it/s]

...

F1 in epoch297 : 0.8708


100%|█████████▉| 299/300 [01:35<00:00,  3.16it/s]

F1 in epoch298 : 0.8708


100%|██████████| 300/300 [01:35<00:00,  3.13it/s]

F1 in epoch299 : 0.8708


In [32]:
y_proba = final.cpu().numpy()

resl = []
for i in y_proba:
  a = [x+1 for x in range(len(i)) if i[x] > 0.5]
  for j in range(len(a)):
    if a[j] >=12:
      a[j] = a[j]+1
  resl.append(a)

test_pred = []
for lis in resl:
  a = [str(i) for i in lis]
  test_pred.append(" ".join(a))

output_df = pd.DataFrame(test_pred, columns = ['Labels'])

# The best model of our prediction data on kaggle is svm.
# Select the prediction of svm model for output.
output_df = pd.DataFrame(test_pred, columns = ['Labels'])
output_df.to_csv('/content/Predicted_labels.txt', sep=",", float_format='%d',index_label="ImageID")